In [51]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [52]:
import pandas as pd
from extract import open_driver, close_driver, wait_for_element
from transforms import parse_record, parse_game_date
from selenium.webdriver.common.by import By
from DML import db_connect, close_db_connect
from sql_queries import summary_table_insert
from psycopg2.extras import execute_batch

In [53]:
def download(url, driver):
    driver.get(url)
    root_element = wait_for_element(source=driver, search_by=By.ID, target="root", unique_element=True)
    data_table = wait_for_element(source=root_element, search_by=By.CLASS_NAME, target="rt-table", unique_element=True)
    
    table_headers = get_table_columns(data_table)
    table_data = get_table_rows(data_table, table_headers)
    
    return table_data

In [54]:
def get_table_columns(source):
    table_headers = wait_for_element(source=source, search_by=By.CLASS_NAME, target="tableHeaderDiv", unique_element=True)
    headers = wait_for_element(source=table_headers, search_by=By.CLASS_NAME, target="rt-th")
    header_names = [header.text for header in headers]
    
    return header_names

In [55]:
def get_table_rows(source, headers):
    table_rows = wait_for_element(source=source, search_by=By.CLASS_NAME, target="rt-tbody", unique_element=True)
    rows = wait_for_element(source=table_rows, search_by=By.CLASS_NAME, target="rt-tr")
    table_data = []
    
    for row in rows:
        cells = wait_for_element(source=row, search_by=By.CLASS_NAME, target="rt-td")
        row_values = [cell.text for cell in cells]
        
        values_map = list(zip(headers, row_values))
        print(values_map)
        map_dict = parse_record(values_map)
        
        orderered_data_list = [
            map_dict["Team"],
            map_dict["game_date"],
            map_dict["home_game"],
            map_dict["opponent"],
            map_dict["W"],
            map_dict["L"],
            map_dict["T"],
            map_dict["OT"],
            map_dict["P"],
            map_dict["P%"],
            map_dict["RW"],
            map_dict["ROW"],
            map_dict["S/O Win"],
            map_dict["GF"],
            map_dict["GA"],
            map_dict["PP%"],
            map_dict["PK%"],
            map_dict["Net PP%"],
            map_dict["Net PK%"],
            map_dict["Shots/GP"],
            map_dict["GA/GP"],
            map_dict["FOW%"]
        ]
        
        table_data.append(map_dict)
            
    return table_data

In [56]:
def scrape_page(url):
    driver = open_driver()
    table_data = download(url, driver)
    close_driver(driver)
    
    # call batch upload function
    
    return table_data

In [57]:
# TODO: batch update function

In [58]:
url = "https://www.nhl.com/stats/teams?aggregate=0&reportType=game&seasonFrom=20212022&seasonTo=20212022&dateFromSeason&gameType=2&filter=gamesPlayed,gte,1&sort=points,wins&page=0&pageSize=100"

In [59]:
table_data = scrape_page(url=url)

[('i', '1'), ('Team', 'New Jersey Devils'), ('Game', '2022/04/18@ VGK'), ('GP', '1'), ('W', '1'), ('L', '0'), ('T', '--'), ('OT', '0'), ('P', '2'), ('P%', '1.000'), ('RW', '1'), ('ROW', '1'), ('S/O Win', '0'), ('GF', '3'), ('GA', '2'), ('GF/GP', '3.00'), ('GA/GP', '2.00'), ('PP%', '0.0'), ('PK%', '100.0'), ('Net PP%', '0.0'), ('Net PK%', '100.0'), ('Shots/GP', '28.0'), ('SA/GP', '44.0'), ('FOW%', '49.2')]


KeyError: 'FOW'

In [ ]:
table_data